In [1]:
# import json
# import datetime

# import geopandas
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import plotly
# import plotly.express as px


# Load Data

In [2]:
# school = geopandas.read_file("../data/geo/nyc_school_districts.geojson")
# census = geopandas.read_file("../data/geo/nyc_2010_tracts_formatted.geojson")
# nyc = pd.read_csv("../data/raw/nyc_evictions_geocoded.csv")

# Preprocess

In [3]:
# nyc.columns = nyc.columns.str.lower()

In [4]:
# nyc[['lon', 'lat']] = nyc.lon_lat.str.split(',', expand=True)
# nyc[['eviction_address','lon_lat','lon','lat']].head()

In [5]:
# nyc['executed_date'] = pd.to_datetime(nyc['executed_date'])
# nyc['executed_year'] = nyc['executed_date'].dt.year
# nyc['executed_month'] = nyc['executed_date'].dt.to_period('M')
# # nyc['executed_week'] = nyc['executed_date'].dt.isocalendar().week

# Outliers

In [6]:
# nyc[nyc['executed_year'] == 2070]

In [7]:
# nyc = nyc[nyc['executed_year'] != 2070].reset_index(drop=True)

# Convert `nyc` to GeoDataFrame

In [8]:
# nyc_gdf = geopandas.GeoDataFrame(nyc, geometry=geopandas.points_from_xy(nyc.lon, nyc.lat, crs=census.crs))

# School Level Visualization

In [9]:
# nyc_school = geopandas.sjoin(nyc_gdf, school, op='within',how='left')

In [10]:
# nyc_school_total = nyc_school.groupby(['school_dist']).size().reset_index()
# nyc_school_total.columns = ['school_dist', 'num_evictions']

In [11]:
# with open('../data/geo/nyc_school_districts.geojson') as json_file:
#     sdGeoJson = json.load(json_file)

In [12]:
# fig = px.choropleth_mapbox(nyc_school_total, geojson=sdGeoJson, color="num_evictions",
#                    color_discrete_sequence = px.colors.qualitative.Dark24,
#                    locations="school_dist", featureidkey="properties.school_dist",opacity=0.5)

# fig.update_layout(
#     height=800,
#     title="Accumulative Number of Evictions by School District since 2017",
#     mapbox_style="streets", 
#     mapbox_accesstoken='pk.eyJ1IjoiZGdpbGxlbiIsImEiOiJjam85OGFvaXIxZXRlM2tubG8zY3E0OHh1In0.KkjAoFhjOOFjXAEuZ1IRog',
#     mapbox_zoom=9, 
#     mapbox_center = {"lat": 40.75, "lon": -73.9}
# )

# fig.show()

In [13]:
# nyc_school_year = nyc_school.groupby(['school_dist', 'executed_year']).size().reset_index()
# nyc_school_year.columns = ['school_dist', 'executed_year', 'num_evictions']

In [14]:
# fig = px.choropleth_mapbox(
#     nyc_school_year, geojson=sdGeoJson, color="num_evictions",
#     color_discrete_sequence = px.colors.qualitative.Dark24, range_color=[0, 2000],
#     locations="school_dist", featureidkey="properties.school_dist", opacity=0.5,
#     animation_frame='executed_year')

# fig.update_layout(
#     height=800,
#     title="Yearly Number of Evictions by School District",
#     mapbox_style="streets", 
#     mapbox_accesstoken='pk.eyJ1IjoiZGdpbGxlbiIsImEiOiJjam85OGFvaXIxZXRlM2tubG8zY3E0OHh1In0.KkjAoFhjOOFjXAEuZ1IRog',
#     mapbox_zoom=9, 
#     mapbox_center = {"lat": 40.75, "lon": -73.9}
# )

# fig.show()

In [15]:
# nyc_school_month = nyc_school.groupby(['school_dist', 'executed_month']).size().reset_index()
# nyc_school_month['executed_month'] = nyc_school_month['executed_month'].astype(str)
# nyc_school_month.columns = ['school_dist', 'executed_month', 'num_evictions']

In [16]:
# fig = px.choropleth_mapbox(
#     nyc_school_month, geojson=sdGeoJson, color="num_evictions",
#     color_discrete_sequence = px.colors.qualitative.Dark24, range_color=[0, 250],
#     locations="school_dist", featureidkey="properties.school_dist", opacity=0.5,
#     animation_frame='executed_month')

# fig.update_layout(
#     height=800,
#     title="Monthly Number of Evictions by School District",
#     mapbox_style="streets", 
#     mapbox_accesstoken='pk.eyJ1IjoiZGdpbGxlbiIsImEiOiJjam85OGFvaXIxZXRlM2tubG8zY3E0OHh1In0.KkjAoFhjOOFjXAEuZ1IRog',
#     mapbox_zoom=9, 
#     mapbox_center = {"lat": 40.75, "lon": -73.9}
# )

# fig.show()

# Time-Series Analysis

In [17]:
# from gluonts.dataset.common import ListDataset
# from gluonts.dataset.util import to_pandas
# from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
# from gluonts.mx.trainer import Trainer
# from gluonts.evaluation.backtest import make_evaluation_predictions

In [18]:
# # Generate time-series df
# nyc_school_month['school_dist'] = nyc_school_month['school_dist'].astype(int)
# table = nyc_school_month.pivot(index='school_dist', columns='executed_month', values='num_evictions')
# table = table[table.columns[:-2]]
# table = table.T

# table.head()

In [19]:
# # Generate train & test set
# train_set = [{
#     'start': pd.Timestamp('2017-01', freq='M'),
#     'target': table[i].values[:35],
#     'item_id': i,
# } for i in range(1, 33)]

# test_set = [{
#     'start': pd.Timestamp('2017-01', freq='M'),
#     'target': table[i].values,
#     'item_id': i,
# } for i in range(1, 33)]

# train_set = ListDataset(train_set, freq='M')
# test_set = ListDataset(test_set, freq='M')

In [20]:
# train_entry = next(iter(train_set))
# test_entry = next(iter(test_set))

# train_series = to_pandas(train_entry)
# test_series = to_pandas(test_entry)

# fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(15, 8))
# fig.suptitle(f"School District {train_entry['item_id']}", fontsize=16)

# train_series.plot(ax=ax[0])
# ax[0].grid(which="both")
# ax[0].legend(["train series"], loc="upper left")

# test_series.plot(ax=ax[1])
# ax[1].axvline(train_series.index[-1], color='r') # end of train dataset
# ax[1].grid(which="both")
# ax[1].legend(["test series", "end of train series"], loc="upper left")

# plt.show()

In [21]:
# estimator = SimpleFeedForwardEstimator(
#     num_hidden_dimensions=[10],
#     prediction_length=4,
#     freq='M',
#     trainer=Trainer(ctx="cpu",
#                     epochs=10,
#                     learning_rate=1e-3,
#                     hybridize=False,
#                     num_batches_per_epoch=100
#                    )
# )

In [22]:
# predictor = estimator.train(train_set)

In [23]:
# forecast_it, ts_it = make_evaluation_predictions(
#     dataset=test_set,  # test dataset
#     predictor=predictor,  # predictor
#     num_samples=100,  # number of sample paths we want for evaluation
# )

In [24]:
# forecasts = list(forecast_it)
# tss = list(ts_it)

In [25]:
# # first entry of the time series list
# ts_entry = tss[0]

# # first entry of the forecast list
# forecast_entry = forecasts[0]
# print(f"Number of sample paths: {forecast_entry.num_samples}")
# print(f"Dimension of samples: {forecast_entry.samples.shape}")
# print(f"Start date of the forecast window: {forecast_entry.start_date}")
# print(f"Frequency of the time series: {forecast_entry.freq}")

In [26]:
# print(f"Mean of the future window:\n {forecast_entry.mean}")
# print(f"0.5-quantile (median) of the future window:\n {forecast_entry.quantile(0.5)}")

In [27]:
# def plot_prob_forecasts(ts_entry, forecast_entry):
#     plot_length = 150
#     prediction_intervals = (50.0, 90.0)
#     legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

#     fig, ax = plt.subplots(1, 1, figsize=(10, 7))
#     ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
#     forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
#     plt.grid(which="both")
#     plt.legend(legend, loc="upper left")
#     plt.show()
    

# plot_prob_forecasts(ts_entry, forecast_entry)